In [1]:
from pymongo import MongoClient
client = MongoClient()
db = client.whosampled
import numpy as np
import pandas as pd
import re

import sys
np.set_printoptions(threshold=sys.maxsize)

from turn_db_main_into_utility_matrix import from_mongo_collection_to_utility_matrix

In [2]:
utility_to_song, utility_to_artist, df = from_mongo_collection_to_utility_matrix(db.main_redo)

In [3]:
# How sparse is matrix?

matrix_size = sparse_artists_util.shape[0]*sparse_artists_util.shape[1] # Number of possible interactions in the matrix
num_purchases = len(sparse_artists_util.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

#We have a matrix that is 99.8% sparse. Kind of a problem. According to https://jessesw.com/Rec-System/ 
# maximum is 99.5%

#If we go by artist, we get down to 99.765%

99.76744013792403


There are 15809 unique records. There are 1324 producers. There are 7789 songs sampled.

In [3]:
from first_recommender_using_implict import get_model_from_utility_matrix
model, user_items = get_model_from_utility_matrix(utility_to_artist)

ImportError: cannot import name 'get_model_from_utility_matrix' from 'first_recommender_using_implict' (/Users/jesse/Documents/recordmender/first_recommender_using_implict.py)

In [8]:
recommendations = model.recommend(1000, user_items)

In [9]:
recommendations

[(3461, 0.029701818),
 (3006, 0.018200487),
 (2665, 0.01734165),
 (1344, 0.016115244),
 (2683, 0.0144840125),
 (3160, 0.014033169),
 (2407, 0.013821856),
 (1693, 0.01372318),
 (2121, 0.0136561245),
 (73, 0.01353282)]

In [12]:
related = model.similar_items(itemid = 1000)

In [13]:
related

[(1000, 0.22357306),
 (3796, 0.19498816),
 (2897, 0.18157251),
 (959, 0.17781958),
 (1173, 0.17781958),
 (36, 0.17781956),
 (1023, 0.17446846),
 (3543, 0.17357084),
 (16, 0.17357084),
 (3039, 0.17357084)]

In [ ]:
from ItemItemRecommender import ItemItemRecommender
recommend = ItemItemRecommender()
recommend.fit(data_sparse)


In [127]:
recomenders_for_4th_Disciple = recommend.pred_one_user(11)

In [137]:
ind = np.argwhere(recomenders_for_4th_Disciple)
data.columns[ind]

Index(['It's a New Day', 'Change the Beat (Female Version)', 'Rubber Jam',
       'Makin' Believe That It's You', 'In the Rain', 'Midnight Theme',
       'The Epic of Bitter Triumph', 'N.T.',
       'Love Theme From "Romeo and Juliet" (A Time for Us)', 'Do It Again',
       ...
       'Think Big', 'The Eye of a Needle', 'One Less Bell to Answer',
       'Don't Say Goodnight (It's Time for Love) (Parts 1 & 2)',
       'The Tragedy (Don't Do It)', 'Ain't No Half Steppin'',
       'Kool It (Here Comes the Fuzz)', 'Trilogy of Terror', 'L Dopa',
       'Do It Your Way'],
      dtype='object', length=1011)

In [ ]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit

# Create a numeric user_id and artist_id column
data['user'] = data['user'].astype("category")
data['artist'] = data['artist'].astype("category")
data['user_id'] = data['user'].cat.codes
data['artist_id'] = data['artist'].cat.codes

# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['plays'].astype(float), (data['artist_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['plays'].astype(float), (data['user_id'], data['artist_id'])))

# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)


#---------------------
# FIND SIMILAR ITEMS
#---------------------

# Find the 10 most similar to Jay-Z
item_id = 147068 #Jay-Z
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(item_id, n_similar)

# Print the names of our most similar artists
for item in similar:
    idx, score = item
    print data.artist.loc[data.artist_id == idx].iloc[0]

    
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 2025
user_id = 2025

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item)

artists = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
    artists.append(data.artist.loc[data.artist_id == idx].iloc[0])
    scores.append(score)

# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'artist': artists, 'score': scores})

print recommendations
